In [6]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from collections import Counter

### Załadowanie danych i okreslenie zmiennych x i y

In [7]:
iris = datasets.load_iris()
X=iris.data
y=iris.target

In [8]:
for keys in iris.keys() :
    print(keys)

data
target
frame
target_names
DESCR
feature_names
filename
data_module


### Podział danych na testowe i treningowe (70% traningowe)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Wersja podstawowa KNN

In [44]:
class KNN():
    def __init__(self, k=3):
        self.k = k
        #inicjalizacja listy przechowującej prognozy 
        self.predictions = []

    @staticmethod
    def euklidesowy(x1, x2):
        # oblicza dystans euklidesowy między 2 pounktami
        return np.sqrt(np.sum((x1-x2)**2))
        
    def fit(self, X_train, y_train):
        #zapisanie wartosci treningowych do wlasciwosci klasy 
        self.X= X_train
        self.y = y_train
        
    @staticmethod
    def najczestszy(lista):
        # funkcja oblicza najczesciej wystepujaca wartosc w liscie 
        hash = {}
        for i in lista:
            if i in hash:
                hash[i] += 1
            else:
                hash[i] = 1     
                
        max_ilosc = 0
        najczestsza = None
        for label, ilosc in hash.items():
            if ilosc > max_ilosc:
                max_ilosc = ilosc
                najczestsza = label
        return najczestsza
        

    def predict(self, test):
        for i in test:
            # obliczanie wszystkich odleglosci miedzy punktem w zbiorze testowym a wszystkimi punktami w zbiorze treningowym
            odleglosci = [self.euklidesowy(i, x) for x in self.X]

            # wybranie indeksow  k najblizszych punktow 
            indeksy = np.argsort(odleglosci)[:self.k]

            # zapisanie klas dla wybranych indeksow w liscie 
            k_wartosci = [self.y[i] for i in indeksy]

            #zapisanie najczesciej wystepujacej wartosci w liscie do listy z prognozami
            self.predictions.append(self.najczestszy(k_wartosci))
            
        return self.predictions
            
                




### Dopasowanie danych do modelu

In [45]:
knn = KNN()
knn.fit(X_train, y_train)

In [46]:
prognozy = knn.predict(X_test)

### Obliczanie dokładności

In [47]:
np.sum(prognozy == y_test) / len(y_test)

1.0

# KNN z 3 Metrykami

In [66]:
from numpy.linalg import norm

class KNN2():
    def __init__(self, k=3, metryka='euklides'):
        self.k = k
        #inicjalizacja listy przechowującej prognozy 
        self.metryka = metryka

    @staticmethod
    def euklidesowy(x1, x2):
        # oblicza dystans euklidesowy między 2 pounktami
        return np.sqrt(np.sum((x1-x2)**2))
    
    @staticmethod
    def manhattan(x1, x2):
        return np.abs(np.sum(x1-x2))

    @staticmethod
    def cosine(x1, x2):
       return np.dot(x1,x2)/(norm(x1)*norm(x2))    
        
    def fit(self, X_train, y_train):
        #zapisanie wartosci treningowych do wlasciwosci klasy 
        self.X= X_train
        self.y = y_train
        
    @staticmethod
    def najczestszy(lista):
        # funkcja oblicza najczesciej wystepujaca wartosc w liscie 
        hash = {}
        for i in lista:
            if i in hash:
                hash[i] += 1
            else:
                hash[i] = 1     
                
        max_ilosc = 0
        najczestsza = None
        for label, ilosc in hash.items():
            if ilosc > max_ilosc:
                max_ilosc = ilosc
                najczestsza = label
        return najczestsza
        

    def predict(self, test):
        predictions = []
        for i in test:
            # obliczanie wszystkich odleglosci miedzy punktem w zbiorze testowym a wszystkimi punktami w zbiorze treningowym
            if self.metryka == 'euklides':
                odleglosci = [self.euklidesowy(i, x) for x in self.X]
                
            if self.metryka == 'manhattan':
                odleglosci = [self.manhattan(i, x) for x in self.X]

            if self.metryka == 'cosinus':
                odleglosci = np.dot(self.X, i)/(norm(self.X, axis=1)*norm(i))

            # wybranie indeksow  k najblizszych punktow 
            indeksy = np.argsort(odleglosci)[:self.k]
            
            # zapisanie klas dla wybranych indeksow w liscie 
            k_wartosci = [self.y[i] for i in indeksy]

            #zapisanie najczesciej wystepujacej wartosci w liscie do listy z prognozami
            predictions.append(self.najczestszy(k_wartosci))
            
        return predictions
            

In [149]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)

In [150]:
knn2 = KNN2(metryka='euklides', k = 3)
knn2.fit(X_train, y_train)

In [151]:
prognozy3 = knn2.predict(X_test)

In [152]:
np.sum(prognozy3 == y_test)/ len(y_test)

0.95

# Wersja Wazona

In [24]:
class KNN_wazony():
    def __init__(self, k=3):
        self.k = k
        #inicjalizacja listy przechowującej prognozy 
        self.predictions = []

    @staticmethod
    def euklidesowy(x1, x2):
        # oblicza dystans euklidesowy między 2 pounktami
        return np.sqrt(np.sum((x1-x2)**2))
        
    def fit(self, X_train, y_train):
        #zapisanie wartosci treningowych do wlasciwosci klasy 
        self.X= X_train
        self.y = y_train
        
    @staticmethod
    def najczestszy_wazony(self,odleglosci, indeksy):
        # funkcja oblicza najczesciej wystepujaca wartosc w liscie 
        wagi = {}
        for i in indeksy:
            label = self.y[i]
            odleglosc = odleglosci[i]
            waga = 1 / (odleglosc + 1e-5)
            if label in wagi:
                wagi[label] += waga
            else:
                wagi[label] = waga
                
        return max(wagi, key=wagi.get)
        

    def predict(self, test):
        for i in test:
            # obliczanie wszystkich odleglosci miedzy punktem w zbiorze testowym a wszystkimi punktami w zbiorze treningowym
            odleglosci = [self.euklidesowy(i, x) for x in self.X]

            # wybranie indeksow  k najblizszych punktow 
            indeksy = np.argsort(odleglosci)[:self.k]

            predykcje = self.najczestszy_wazony(self, odleglosci, indeksy)
            
            #zapisanie najczesciej wystepujacej wartosci w liscie do listy z prognozami
            self.predictions.append(predykcje)
            
        return self.predictions
            
                


In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)

In [26]:
knn_wazony = KNN_wazony(k = 3)
knn_wazony.fit(X_train, y_train)

In [27]:
prognozy = knn_wazony.predict(X_test)

In [28]:
np.sum(prognozy == y_test)/ len(y_test)

0.9666666666666667

In [ ]:
-